In [1]:
import sys
sys.path.append(r'C:\Users\Mi\Desktop\code\git\python-utils-ak')
from utils_ak.interactive_imports import *

def _import_globals(module):
    for x in dir(module):
        if not x.startswith('_'):
            globals()[x] = getattr(module, x)
            
def _load(module_obj, module_name=None, reload=False, import_globals=False):
    module = load(module_obj, module_name, reload, import_globals=False)
    if import_globals:
        _import_globals(module)
    return module

def load_sources():
    _load('src/time.py', import_globals=True, reload=True)
    _load('src/block.py', import_globals=True, reload=True)
    _load('src/color.py', import_globals=True, reload=True)
    _load('src/drawing.py', import_globals=True, reload=True)
    _load('src/interval.py', import_globals=True, reload=True)

In [2]:
load_sources()

# Models

In [3]:
import sys
# sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/2020.10-umalat/umalat')
sys.path.append(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat')
from app.models import *
from app.models import Boiling as BoilingModel
from app.models import Termizator as TermizatorModel

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# sqlite_filepath = '/home/akadaner/Desktop/data.sqlite'
sqlite_filepath = r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite"
engine = create_engine(f"sqlite:///{sqlite_filepath}")
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

def cast_sku(obj):
    if isinstance(obj, SKU):
        return obj
    elif isinstance(obj, (str, int)):
        obj = str(obj)
        return session.query(SKU).filter(SKU.id == obj).first()
    else:
        raise Exception('Unknown sku type')
        
def cast_boiling(obj):
    if isinstance(obj, Boiling):
        return obj
    elif isinstance(obj, (str, int)):
        obj = str(obj)
        return session.query(Boiling).filter(Boiling.id == obj).first()
    else:
        raise Exception('Unknown boiling type')

In [4]:
values = session.query(SKU).all()
skus = pd.DataFrame(values, columns=['sku'])
skus['name'] = skus['sku'].apply(lambda sku:sku.name)
skus['percent'] = skus['sku'].apply(lambda sku:sku.boiling.percent)
skus.head()

,sku,name,percent
0,<SKU 1>,"Моцарелла ""Pretto"" (для бутербродов), 45%, 0,2 кг, т/ф, (9 шт)",2.7
1,<SKU 2>,"Моцарелла ""Pretto"", 45%, 1,2 кг, в/у",2.7
2,<SKU 3>,"Моцарелла ""Unagrande"", 45%, 0,12 кг, ф/п (кубики)",2.7
3,<SKU 4>,"Моцарелла ""Unagrande"", 45%, 3 кг, пл/л",2.7
4,<SKU 5>,"Моцарелла (палочки), 45%, кг, пл/л",2.7


In [5]:
values = session.query(Boiling).all()
boilings = pd.DataFrame(values, columns=['boiling'])
boilings['percent'] = boilings['boiling'].apply(lambda b: b.percent)
boilings

,boiling,percent
0,<Boiling 1>,2.7
1,<Boiling 2>,2.7
2,<Boiling 3>,2.7
3,<Boiling 4>,2.7
4,<Boiling 5>,3.3
5,<Boiling 6>,3.3
6,<Boiling 7>,3.3
7,<Boiling 8>,3.3
8,<Boiling 9>,3.6
9,<Boiling 10>,3.6


In [6]:
TERMIZATOR = session.query(Termizator).first()
TERMIZATOR.pouring_time = 30

In [304]:
def get_sample_boiling_conf(boiling_type='water'):
    if boiling_type == 'water':
        tmp = skus[skus['percent'] == 3.3]
        sku = tmp.iloc[0]['sku']
        BOILING_CONF = sku.boiling
        BOILING_REQUEST = {sku: 850}
        return BOILING_CONF, BOILING_REQUEST
    elif boiling_type == 'salt':
        tmp = skus[skus['percent'] == 2.7]
        sku = tmp.iloc[0]['sku']
        BOILING_CONF = sku.boiling
        BOILING_REQUEST = {sku: 850}
        return BOILING_CONF, BOILING_REQUEST

In [307]:
def make_melting_and_packing(boiling_conf, boiling_request, boiling_type, 
                            melting_line=None):
    maker = BlockMaker(default_push_func=dummy_push)
    make = maker.make
    
    with make('melting_and_packing'):
        # packing and melting time
        packing_times = []
        if boiling_type == 'water':
            melting_time = custom_round(850 / boiling_conf.meltings.speed * 60 * 1.3, 5, rounding='ceil')

            # todo: make proper order
            for sku, sku_kg in boiling_request.items():
                packing_speed = min(sku.packing_speed, boiling_conf.meltings.speed)
                packing_times.append(custom_round(sku_kg / packing_speed * 60, 5, rounding='ceil'))
            total_packing_time = sum(packing_times) + (len(packing_times) - 1) * 5 # add time for reconfiguration - 5 minutes between each
            full_melting_time = boiling_conf.meltings.serving_time + melting_time
            
        elif boiling_type == 'salt':
            # todo: make proper order
            for sku, sku_kg in boiling_request.items():
                packing_times.append(custom_round(sku_kg / sku.packing_speed * 60, 5, rounding='ceil'))
            total_packing_time = sum(packing_times) + (len(packing_times) - 1) * 5 # add time for reconfiguration - 5 minutes between each
            
            melting_time = total_packing_time
            
            full_melting_time = boiling_conf.meltings.serving_time + melting_time + boiling_conf.meltings.salting_time
        
        # todo: make proper order
        form_factor_label = ' / '.join([str(sku.form_factor) for sku in boiling_request.keys()])
        brand_label = ' / '.join([sku.brand_name for sku in boiling_request.keys()])
        
        with make('melting', y=10, time_size=full_melting_time, melting_line=melting_line):
            with make(y=0):
                make('melting_label', time_size=4 * 5)
                make('melting_name', time_size=full_melting_time - 4 * 5, form_factor_label=form_factor_label)
            with make(y=1):
                make('serving', time_size=boiling_conf.meltings.serving_time)
                make('melting_process', time_size=melting_time, speed=boiling_conf.meltings.speed)

            with make(y=2):
                make(time_size=boiling_conf.meltings.serving_time, visible=False)
                if boiling_type == 'water':
                    make('cooling1', time_size=boiling_conf.meltings.first_cooling_time)
                    make('cooling2', time_size=boiling_conf.meltings.second_cooling_time)
                elif boiling_type == 'salt':
                    make('salting', time_size=boiling_conf.meltings.salting_time)

        # prepare time
        if boiling_type == 'water':
            prepare_time = boiling_conf.meltings.serving_time + boiling_conf.meltings.first_cooling_time + boiling_conf.meltings.second_cooling_time
        elif boiling_type == 'salt':
            prepare_time = boiling_conf.meltings.serving_time + boiling_conf.meltings.salting_time

        with make('packing', t=prepare_time / 5, time_size=total_packing_time, push_func=add_push):
            with make(y=0):
                make('packing_label', time_size=3 * 5)
                make('packing_name', time_size=total_packing_time - 3 * 5, form_factor_label=form_factor_label)
            with make(y=1):
                make('packing_brand', time_size=total_packing_time, brand_label=brand_label)
            with make(y=2):
                make('configuration', time_size=packing_times[0], visible=False)

                for packing_time in packing_times[1:]:
                    make('configuration', size=1)
                    make('configuration', time_size=packing_time, visible=False)

    res = maker.root.children[0]
    res.rel_props['size'] = max(c.end for c in res.children)
    
    return res

BOILING_CONF, BOILING_REQUEST = get_sample_boiling_conf('salt')
block = make_melting_and_packing(BOILING_CONF, BOILING_REQUEST, 'salt', melting_line='0')
print(draw_print(block))

========================================= melting_and_packing (0,41]
========================================= melting (0,41]
==== melting_label (0,4]
    ===================================== melting_name (4,41]
====== serving (0,6]
      ========== melting_process (6,16]
====== block (0,6]
      ========================= salting (6,31]
                               ========== packing (31.0,41.0]
                               === packing_label (0,3]
                                  ======= packing_name (3,10]
                               ========== packing_brand (0,10]
                               ========== configuration (0,10]



In [320]:
def make_boiling(boiling_conf, boiling_request, boiling_type='water', block_num=12, pouring_line=None, melting_line=None):
    maker = BlockMaker(default_push_func=dummy_push)
    make = maker.make
    
    boiling_label = '{} {} {} 8000кг'.format(boiling_conf.percent, boiling_conf.ferment, 'с лактозой' if boiling_conf.is_lactose else 'безлактозная')


    with make('boiling', block_num=block_num, boiling_type=boiling_type, boiling_label=boiling_label):
        # todo: make boiling size

        timings = []
        timings.append(boiling_conf.pourings.pouring_time)
        timings.append(boiling_conf.pourings.soldification_time)
        timings.append(boiling_conf.pourings.cutting_time)
        timings.append(boiling_conf.pourings.pouring_off_time)
        timings.append(boiling_conf.pourings.extra_time)
        
        with make('pouring', time_size=sum(timings), pouring_line=pouring_line):
            with make(y=0):
                make('termizator', time_size=TERMIZATOR.pouring_time)
                make('pouring_name', time_size=sum(timings) - TERMIZATOR.pouring_time)
            with make(y=1):
                make('pouring_and_fermenting', time_size=timings[0])
                make('soldification', time_size=timings[1])
                make('cutting', time_size=timings[2])
                make('pouring_off', time_size=timings[3])
                make('extra', time_size=timings[4])

        # todo: specify parameter
        make('drenator', size=cast_t('03:50'), visible=False)
        
        make(make_melting_and_packing(boiling_conf, boiling_request, boiling_type, melting_line=melting_line))
        

    res = maker.root.children[0]
    res.rel_props['size'] = max(c.end for c in res.children)

    return res

BOILING_CONF, BOILING_REQUEST = get_sample_boiling_conf('salt')
block = make_boiling(BOILING_CONF, BOILING_REQUEST, 'salt', pouring_line='0', melting_line='0')
print(draw_print(block))

================================================================================================================== boiling (0,114]
=========================== pouring (0,27]
====== termizator (0,6]
      ===================== pouring_name (6,27]
============ pouring_and_fermenting (0,12]
            ====== soldification (12,18]
                  ==== cutting (18,22]
                      === pouring_off (22,25]
                         == extra (25,27]
                           ============================================== drenator (27,73]
                                                                         ========================================= melting_and_packing (73,114]
                                                                         ========================================= melting (0,41]
                                                                         ==== melting_label (0,4]
                                                                             ===

In [321]:
def make_termizator_cleaning_block(cleaning_type):
    maker = BlockMaker(default_push_func=dummy_push)
    make = maker.make
    
    with make('cleaning'):
        if cleaning_type == 'short':
            make('short_cleaning', t=0, size=8)
        elif cleaning_type == 'full':
            make('full_cleaning', t=0, size=16)
    
    res = maker.root.children[0]
    res.rel_props['size'] = max(c.end for c in res.children)
    return res

# Drawing

In [310]:
root = Block('root', t=cast_t('01:45'))
BOILING_CONF, BOILING_REQUEST = get_sample_boiling_conf('salt')
block = make_boiling(BOILING_CONF, BOILING_REQUEST, 'salt', melting_line='0', pouring_line='1')
dummy_push(root, block)
draw_schedule(root, STYLE, 'output.xlsx', init_sheet)

termizator 9 (14,20] (21,27]
pouring_name 9 (20,41] (27,48]
pouring_and_fermenting 10 (14,26] (21,33]
soldification 10 (26,32] (33,39]
cutting 10 (32,36] (39,43]
pouring_off 10 (36,39] (43,46]
extra 10 (39,41] (46,48]
melting_label 33 (17,21] (96,100]
melting_name 33 (21,58] (100,137]
serving 34 (17,23] (96,102]
melting_process 34 (23,33] (102,112]
salting 35 (23,48] (102,127]
packing_label 46 (48.0,51.0] (127.0,130.0]
packing_name 46 (51.0,58.0] (130.0,137.0]
packing_brand 47 (48.0,58.0] (127.0,137.0]


PermissionError: [Errno 13] Permission denied: 'output.xlsx'

# Schedule

In [311]:
def load_style():
    BASE_STYLE = {
        'pouring': {'beg_time': '01:00'},
        'termizator': {'text': '{block_num} налив'},
        'pouring_name': {'text': '{boiling_label}'},
        'pouring_and_fermenting': {'text': 'налив/внесение\nзакваски', 'color': (255, 192, 0)}, # orange color
        'soldification': {'text': 'схватка', 'color': 'yellow'},
        'cutting': {'text': 'резка/обсушка', 'color': '#92d050'}, # light green color
        'pouring_off': {'text': 'слив', 'color': 'red'},
        'melting': {'beg_time': '07:00'},
        'melting_label': {'text': '{block_num}'},
        'melting_name': {'text': '{form_factor_label}'},
        'serving': {'color': 'orange', 'text': 'подача и вымешивание'},
        'melting_process': {'color': 'orange', 'text': lambda b: 'плавление/формирование {speed} кг/ч' if b.abs_props['boiling_type'] == 'water' else 'плавление/формирование'},
        'cooling1': {'text': 'охлаж1 <{size} * 5> мин'},
        'cooling2': {'text': 'охлаж2 <{size} * 5> мин'},
        'packing': {'beg_time': '07:00'},
        'packing_label': {'text': '{block_num}'},
        'packing_name': {'text': '{form_factor_label}'},
        'packing_brand': {'color': '#f2dcdb', 'text': '{brand_label}'}, # pink color
        'configuration': {'color': 'red'},
        'salting': {'color': (0, 176, 240), 'text': 'посолка'}, # light-blue color
        'cleaning': {'beg_time': '01:00', 'h': 2},
        'full_cleaning': {'color': 'yellow', 'text': 'полная мойка термизатора'},
        'short_cleaning': {'color': 'yellow', 'text': 'короткая мойка\nтермизатора'},
    }


    ROW_STYLE_DEV = {
        'pouring': {'y': lambda b: {'0': 1, '1': 3, '2': 5, '3': 7}[b.abs_props['pouring_line']]},
        'melting': {'y': lambda b: 9 if b.abs_props['boiling_type'] == 'water' else {'0': 14, '1': 17, '2': 20, '3': 23}[b.abs_props['melting_line']]},
        'packing': {'y': lambda b: 12 if b.abs_props['boiling_type'] == 'water' else 26},
        'cleaning': {'y': 28}
    }

    ROW_STYLE_PROD = {
        'pouring': {'y': lambda b: {'0': 6, '1': 9, '2': 15, '3': 18}[b.abs_props['pouring_line']]},
        'melting': {'y': lambda b: 24 if b.abs_props['boiling_type'] == 'water' else {'0': 33, '1': 36, '2': 39, '3': 42}[b.abs_props['melting_line']]},
        'packing': {'y': lambda b: 29 if b.abs_props['boiling_type'] == 'water' else 46},
        'cleaning': {'y': 12}
    }

    ROW_STYLE = ROW_STYLE_DEV if MODE == 'dev' else ROW_STYLE_PROD

    styles = [BASE_STYLE, ROW_STYLE]

    STYLE = {}
    for style in styles:
        for k, v in style.items():
            if k not in STYLE:
                STYLE[k] = v
            else:
                STYLE[k].update(v)
    return STYLE
STYLE = load_style()

In [312]:
def boiling_validator(parent, boiling):
    if not parent.children:
        return
    # todo: optimize
    #     b1 = max(parent.children, key=lambda b: b.rel_props.get('t', 0))

    for b1 in parent.children[-4:]:
        b2 = boiling

        b1.rel_props['props_mode'] = 'absolute'
        b2.rel_props['props_mode'] = 'absolute'

        b1.upd_abs_props()
        b2.upd_abs_props()

        validate_disjoint(b1['pouring'][0]['termizator'], b2['pouring'][0]['termizator'])
        
        if b1['pouring'].props['pouring_line'] == b2['pouring'].props['pouring_line']:
                validate_disjoint(b1['pouring'], b2['pouring'])
        
        if (boiling_type := b1.props['boiling_type']) == b2.props['boiling_type']:
            if boiling_type == 'water':
                validate_disjoint(b1['melting_and_packing']['melting'], b2['melting_and_packing']['melting'])
            else:
                if b1['melting_and_packing']['melting'].props['melting_line'] == b2['melting_and_packing']['melting'].props['melting_line']:
                    validate_disjoint(b1['melting_and_packing']['melting'], b2['melting_and_packing']['melting'])
                else:
                    validate_disjoint(b1['melting_and_packing']['melting'][1]['melting_process'], b2['melting_and_packing']['melting'][1]['melting_process'])

            validate_disjoint(b1['melting_and_packing']['packing'], b2['melting_and_packing']['packing'])

        b1.rel_props.pop('props_mode')
        b2.rel_props.pop('props_mode')

        b1.upd_abs_props()
        b2.upd_abs_props()

In [295]:
from itertools import product

iter_water_props = [{'pouring_line': str(v)} for v in [0, 1]]
iter_salt_props = [{'pouring_line': str(v[0]), 'melting_line': str(v[1])} for v in product([1, 2, 3], [0, 1, 2, 3])]
iter_water_props, iter_salt_props

([{'pouring_line': '0'}, {'pouring_line': '1'}],
 [{'pouring_line': '1', 'melting_line': '0'},
  {'pouring_line': '1', 'melting_line': '1'},
  {'pouring_line': '1', 'melting_line': '2'},
  {'pouring_line': '1', 'melting_line': '3'},
  {'pouring_line': '2', 'melting_line': '0'},
  {'pouring_line': '2', 'melting_line': '1'},
  {'pouring_line': '2', 'melting_line': '2'},
  {'pouring_line': '2', 'melting_line': '3'},
  {'pouring_line': '3', 'melting_line': '0'},
  {'pouring_line': '3', 'melting_line': '1'},
  {'pouring_line': '3', 'melting_line': '2'},
  {'pouring_line': '3', 'melting_line': '3'}])

In [296]:
load_sources()

In [297]:
# draw salt and water
root = Block('root', t=cast_t('01:45'))

boilings = []

def gen_boiling(boiling_type):
    BOILING_CONF, BOILING_REQUEST = get_sample_boiling_conf(boiling_type)
    return make_boiling(BOILING_CONF, BOILING_REQUEST, boiling_type)

for i in range(2):
    boilings.append(gen_boiling('water'))
    
for i in range(2):
    boilings.append(gen_boiling('salt'))

for b in boilings:
    iter_props = iter_water_props if b.props['boiling_type'] == 'water' else iter_salt_props
    dummy_push(root, b, iter_props=iter_props, validator=boiling_validator, beg='last_beg', max_tries=100)

draw_schedule(root, STYLE, 'output.xlsx', init_sheet)
root

termizator 6 (14,20] (21,27]
pouring_name 6 (20,40] (27,47]
pouring_and_fermenting 7 (14,22] (21,29]
soldification 7 (22,28] (29,35]
cutting 7 (28,35] (35,42]
pouring_off 7 (35,38] (42,45]
extra 7 (38,40] (45,47]
melting_label 24 (16,20] (95,99]
melting_name 24 (20,37] (99,116]
serving 25 (16,22] (95,101]
melting_process 25 (22,37] (101,116]
cooling1 26 (22,27] (101,106]
cooling2 26 (27,37] (106,116]
packing_label 29 (37.0,40.0] (116.0,119.0]
packing_name 29 (40.0,49.0] (119.0,128.0]
packing_brand 30 (37.0,49.0] (116.0,128.0]
termizator 9 (35,41] (42,48]
pouring_name 9 (41,61] (48,68]
pouring_and_fermenting 10 (35,43] (42,50]
soldification 10 (43,49] (50,56]
cutting 10 (49,56] (56,63]
pouring_off 10 (56,59] (63,66]
extra 10 (59,61] (66,68]
melting_label 24 (37,41] (116,120]
melting_name 24 (41,58] (120,137]
serving 25 (37,43] (116,122]
melting_process 25 (43,58] (122,137]
cooling1 26 (43,48] (122,127]
cooling2 26 (48,58] (127,137]
packing_label 29 (58.0,61.0] (137.0,140.0]
packing_name

root (21, 21]
  boiling (0, 107]
    pouring (0, 26]
      block (0, 0]
        termizator (0, 6]
        pouring_name (6, 26]
      block (0, 0]
        pouring_and_fermenting (0, 8]
        soldification (8, 14]
        cutting (14, 21]
        pouring_off (21, 24]
        extra (24, 26]
    drenator (26, 74]
    melting_and_packing (74, 107]
      melting (0, 21]
        block (0, 0]
          melting_label (0, 4]
          melting_name (4, 21]
        block (0, 0]
          serving (0, 6]
          melting_process (6, 21]
        block (0, 0]
          block (0, 6]
          cooling1 (6, 11]
          cooling2 (11, 21]
      packing (21.0, 33.0]
        block (0, 0]
          packing_label (0, 3]
          packing_name (3, 12]
        block (0, 0]
          packing_brand (0, 12]
        block (0, 0]
          configuration (0, 12]
  boiling (21, 128]
    pouring (0, 26]
      block (0, 0]
        termizator (0, 6]
        pouring_name (6, 26]
      block (0, 0]
        pouring_and_

# Final Water Algo

In [322]:
import anyconfig
request = anyconfig.load('2020.11.22 sample_day_request.yaml')
date = cast_datetime(request['Date'])

values = []
for boiling in request['Boilings']:
    for sku_id, sku_kg in boiling['SKUVolumes'].items():
        values.append([boiling['BoilingId'], sku_id, sku_kg])
df = pd.DataFrame(values)

values = []
for boiling_id, boiling_grp in df.groupby(0):
    boiling_dic = boiling_grp[[1,2]].set_index(1).to_dict(orient='index')
    boiling_dic = {k: v[2] for k, v in boiling_dic.items()}  # {'34': 1110.0, '35': 0.0, ...}
    total_kg = sum(boiling_dic.values())
    
    # round to get full
    # todo: proper logic
    total_kg = custom_round(total_kg, 850, rounding='floor')
    
    n_boilings = int(total_kg / 850)
    for i in range(n_boilings):
        cur_kg = 850
        boiling_request = {}
        for k, v in list(boiling_dic.items()):
            boil_kg = min(cur_kg, boiling_dic[k])
            
            boiling_dic[k] -= boil_kg
            cur_kg -= boil_kg
            
            if k not in boiling_request:
                boiling_request[k] = 0
            boiling_request[k] += boil_kg
            
            if cur_kg == 0:
                break
                
        if cur_kg != 0:
            # any non-zero
            print('Non-zero')
            k = [k for k, v in boiling_request.items() if v != 0][0]
            boiling_request[k] += cur_kg
        
        boiling_request = {k: v for k, v in boiling_request.items() if v != 0}
        values.append([boiling_id, boiling_request])
df = pd.DataFrame(values, columns=['boiling_id', 'request'])
df['boiling_id'] = df['boiling_id'].astype(str)
df['boiling_type'] = df['boiling_id'].apply(lambda boiling_id: 'salt' if str(cast_boiling(boiling_id).percent) == '2.7' else 'water')
df['used'] = False
df['iter_props'] = df['boiling_type'].apply(lambda boiling_type: iter_water_props if boiling_type == 'water' else iter_salt_props)
df.head()

,boiling_id,request,boiling_type,used,iter_props
0,1,{'34': 850},salt,False,"[{'pouring_line': '1', 'melting_line': '0'}, {'pouring_line': '1', 'melting_line': '1'}, {'pouring_line': '1', 'melting_line': '2'}, {'pouring_line': '1', 'melting_line': '3'}, {'pouring_line': '2', 'melting_line': '0'}, {'pouring_line': '2', 'melting_line': '1'}, {'pouring_line': '2', 'melting_line': '2'}, {'pouring_line': '2', 'melting_line': '3'}, {'pouring_line': '3', 'melting_line': '0'}, {'pouring_line': '3', 'melting_line': '1'}, {'pouring_line': '3', 'melting_line': '2'}, {'pouring_line': '3', 'melting_line': '3'}]"
1,1,"{'34': 260.0, '37': 590.0}",salt,False,"[{'pouring_line': '1', 'melting_line': '0'}, {'pouring_line': '1', 'melting_line': '1'}, {'pouring_line': '1', 'melting_line': '2'}, {'pouring_line': '1', 'melting_line': '3'}, {'pouring_line': '2', 'melting_line': '0'}, {'pouring_line': '2', 'melting_line': '1'}, {'pouring_line': '2', 'melting_line': '2'}, {'pouring_line': '2', 'melting_line': '3'}, {'pouring_line': '3', 'melting_line': '0'}, {'pouring_line': '3', 'melting_line': '1'}, {'pouring_line': '3', 'melting_line': '2'}, {'pouring_line': '3', 'melting_line': '3'}]"
2,1,"{'37': 60.39999999999998, '39': 789.6}",salt,False,"[{'pouring_line': '1', 'melting_line': '0'}, {'pouring_line': '1', 'melting_line': '1'}, {'pouring_line': '1', 'melting_line': '2'}, {'pouring_line': '1', 'melting_line': '3'}, {'pouring_line': '2', 'melting_line': '0'}, {'pouring_line': '2', 'melting_line': '1'}, {'pouring_line': '2', 'melting_line': '2'}, {'pouring_line': '2', 'melting_line': '3'}, {'pouring_line': '3', 'melting_line': '0'}, {'pouring_line': '3', 'melting_line': '1'}, {'pouring_line': '3', 'melting_line': '2'}, {'pouring_line': '3', 'melting_line': '3'}]"
3,1,{'39': 850.0},salt,False,"[{'pouring_line': '1', 'melting_line': '0'}, {'pouring_line': '1', 'melting_line': '1'}, {'pouring_line': '1', 'melting_line': '2'}, {'pouring_line': '1', 'melting_line': '3'}, {'pouring_line': '2', 'melting_line': '0'}, {'pouring_line': '2', 'melting_line': '1'}, {'pouring_line': '2', 'melting_line': '2'}, {'pouring_line': '2', 'melting_line': '3'}, {'pouring_line': '3', 'melting_line': '0'}, {'pouring_line': '3', 'melting_line': '1'}, {'pouring_line': '3', 'melting_line': '2'}, {'pouring_line': '3', 'melting_line': '3'}]"
4,1,{'39': 850.0},salt,False,"[{'pouring_line': '1', 'melting_line': '0'}, {'pouring_line': '1', 'melting_line': '1'}, {'pouring_line': '1', 'melting_line': '2'}, {'pouring_line': '1', 'melting_line': '3'}, {'pouring_line': '2', 'melting_line': '0'}, {'pouring_line': '2', 'melting_line': '1'}, {'pouring_line': '2', 'melting_line': '2'}, {'pouring_line': '2', 'melting_line': '3'}, {'pouring_line': '3', 'melting_line': '0'}, {'pouring_line': '3', 'melting_line': '1'}, {'pouring_line': '3', 'melting_line': '2'}, {'pouring_line': '3', 'melting_line': '3'}]"


In [323]:
def pick(df, boiling_type):
    tmp = df
    tmp = tmp[tmp['used'] == False]
    tmp = tmp[tmp['boiling_type'] == boiling_type]
    
    if len(tmp) == 0:
        return
    
    row = tmp.iloc[0]
    df.loc[row.name, 'used'] = True
    return row


load_sources()
# draw salt and water
root = Block('root')

boilings = []


def make_boiling_row(i, row):
    return make_boiling(cast_boiling(row['boiling_id']), {cast_sku(k): v for k, v in row['request'].items()}, row['boiling_type'], block_num=i + 1)

i, row = 0, pick(df, 'water')
b = make_boiling_row(i, row)
beg = cast_t('08:00') - b['melting_and_packing'].beg
dummy_push(root, b, iter_props=row['iter_props'], validator=boiling_validator, beg=beg, max_tries=100)

i, row = 1, pick(df, 'salt')
b = make_boiling_row(i, row)
beg = cast_t('08:20') - b['melting_and_packing'].beg
dummy_push(root, b, iter_props=row['iter_props'], validator=boiling_validator, beg=beg, max_tries=100)

boiling_types = ['water', 'salt']
cur_type_i = -1
cur_i = 2
while True:
    cur_type_i = (cur_type_i + 1) % 2
    boiling_type = boiling_types[cur_type_i]
    row = pick(df, boiling_type)
    
    # todo: make properly
    if row is None:
        if boiling_type == 'salt':
            break
        else:
            continue
    b = make_boiling_row(cur_i, row)
    dummy_push(root, b, iter_props=row['iter_props'], validator=boiling_validator, beg='last_beg', max_tries=100)
    cur_i += 1
# for i, row in df.iterrows():
#     b = make_boiling(cast_boiling(row['boiling_id']), {cast_sku(k): v for k, v in row['request'].items()}, row['boiling_type'], block_num=i + 1)
#     dummy_push(root, b, iter_props=iter_props, validator=boiling_validator, beg='last_beg', max_tries=100)

In [324]:
# add cleanings
root.rel_props['props_mode'] = 'absolute'
root.upd_abs_props()

boilings = list(root.children)
for i in range(len(boilings) - 1):
    a, b = boilings[i:i + 2]
    a.upd_abs_props()
    b.upd_abs_props()
    rest = b['pouring'][0]['termizator'].beg - a['pouring'][0]['termizator'].end 
#     print(rest,)
    if 12 <= rest < 18:
        cleaning = make_termizator_cleaning_block('short')
        cleaning.rel_props['t'] = b['pouring'][0]['termizator'].beg - cleaning.size
        add_push(root, cleaning)
    elif rest >= 18:
        cleaning = make_termizator_cleaning_block('full')
        cleaning.rel_props['t'] = b['pouring'][0]['termizator'].beg - cleaning.size
        add_push(root, cleaning)
    
    if i == len(boilings) - 2:
        # last one
        cleaning = make_termizator_cleaning_block('full')
        cleaning.rel_props['t'] = b['pouring'][0]['termizator'].end + 1 # add five minutes extra
        add_push(root, cleaning)
        
        
root.rel_props.pop('props_mode')
root.upd_abs_props()

In [325]:
remove('output.xlsx')
load_sources()
MODE = 'prod'
STYLE = load_style()

root.rel_props['size'] = max(c.end for c in root.children)
init_sheet_func = init_sheet if MODE == 'dev' else init_template_sheet
# init_sheet_func = init_sheet if MODE == 'dev' else lambda: init_template_sheet(template_fn=r'2020.10.06 pouring_template.xlsx')
draw_schedule(root, STYLE, '_drawing.xlsx', init_sheet_func)
rename('_drawing.xlsx', 'output.xlsx')

termizator 6 (17,23] (24,30]
pouring_name 6 (23,43] (30,50]
pouring_and_fermenting 7 (17,25] (24,32]
soldification 7 (25,31] (32,38]
cutting 7 (31,38] (38,45]
pouring_off 7 (38,41] (45,48]
extra 7 (41,43] (48,50]
melting_label 24 (17,21] (96,100]
melting_name 24 (21,38] (100,117]
serving 25 (17,23] (96,102]
melting_process 25 (23,38] (102,117]
cooling1 26 (23,28] (102,107]
cooling2 26 (28,38] (107,117]
packing_label 29 (38.0,41.0] (117.0,120.0]
packing_name 29 (41.0,55.0] (120.0,134.0]
packing_brand 30 (38.0,55.0] (117.0,134.0]
configuration 31 (39.0,40.0] (118.0,119.0]
termizator 9 (23,29] (30,36]
pouring_name 9 (29,50] (36,57]
pouring_and_fermenting 10 (23,35] (30,42]
soldification 10 (35,41] (42,48]
cutting 10 (41,45] (48,52]
pouring_off 10 (45,48] (52,55]
extra 10 (48,50] (55,57]
melting_label 33 (24,28] (103,107]
melting_name 33 (28,58] (107,137]
serving 34 (24,30] (103,109]
melting_process 34 (30,40] (109,119]
salting 35 (30,48] (109,127]
packing_label 46 (48.0,51.0] (127.0,130.0

melting_process 40 (280,309] (359,388]
salting 41 (280,298] (359,377]
packing_label 46 (298.0,301.0] (377.0,380.0]
packing_name 46 (301.0,327.0] (380.0,406.0]
packing_brand 47 (298.0,327.0] (377.0,406.0]
configuration 48 (308.0,309.0] (387.0,388.0]
configuration 48 (311.0,312.0] (390.0,391.0]
termizator 9 (302,308] (309,315]
pouring_name 9 (308,329] (315,336]
pouring_and_fermenting 10 (302,314] (309,321]
soldification 10 (314,320] (321,327]
cutting 10 (320,324] (327,331]
pouring_off 10 (324,327] (331,334]
extra 10 (327,329] (334,336]
melting_label 33 (303,307] (382,386]
melting_name 33 (307,344] (386,423]
serving 34 (303,309] (382,388]
melting_process 34 (309,319] (388,398]
salting 35 (309,334] (388,413]
packing_label 46 (334.0,337.0] (413.0,416.0]
packing_name 46 (337.0,344.0] (416.0,423.0]
packing_brand 47 (334.0,344.0] (413.0,423.0]
termizator 15 (312,318] (319,325]
pouring_name 15 (318,339] (325,346]
pouring_and_fermenting 16 (312,324] (319,331]
soldification 16 (324,330] (331,337]